<a href="https://colab.research.google.com/github/cbachen1997/CAUstudy/blob/master/ROI2Patches.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#挂载谷歌云盘
from google.colab import drive
drive.mount('/content/gdrive')
####google云盘授权#####
##每个notebook执行一次###
__author__='CBA'
from google.colab import drive

#增加PyDrive操作库
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#授权登录
auth.authenticate_user()
gauth=GoogleAuth()

In [ ]:
import os
os.chdir("/content/gdrive/MyDrive/CM")
print (os.path.abspath('.')) #获取当前工作目录路径
print(os.listdir('.'))

/content/gdrive/MyDrive/CM
['models.py', 'data_util.py', 'main.py', 'ops.py', 'img', 'roi_patches', 'model.ipynb', 'ROI2Patches.ipynb', 'GetTriplet.ipynb']


In [ ]:
from __future__ import print_function, division
import numpy as np
import tifffile as tiff
from PIL import Image
import codecs
import os
import cv2
import scipy.io as sio
from keras.utils.np_utils import to_categorical
from scipy.cluster.vq import whiten
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf


In [ ]:
#常数设置
r=5#pad半径
save_path='/content/gdrive/MyDrive/CM'

In [ ]:
def read_image(filename):
  img = tiff.imread(filename)
  img = np.asarray(img, dtype=np.float32)
  return img

In [ ]:
def sample_normalization(data):
    """
    normalize each sample to 0-1
    Input:
        sample
    Output:
        Normalized sample
    """
    if np.max(data) == np.min(data):
        return np.ones_like(data, dtype=np.float32) * 1e-6
    else:
        return 1.0 * (data - np.min(data)) / (np.max(data) - np.min(data))

In [ ]:
def save_img_patch(img_patch, classes, i, j,save_path=save_path):
  print(img_patch.shape)
  img_patch_name = 'patch' + '_' + str(i) + '_' + str(j) +'_'+str(classes)+ '.png'
  img_patch_save_path = save_path + '/' + img_patch_name
  cv2.imwrite(img_patch_save_path, img_patch)
  return True

In [ ]:
def create_patches(img_path,roi_img_path):
  lst_img=read_image(img_path)
  train_img=read_image(roi_img_path)
  #pad以防止roi出现在边界
  lst_img=np.pad(lst_img,((r,r),(r,r),(0,0)),'symmetric')
  #灰度图不能直接pad
  train_img=cv2.cvtColor(train_img, cv2.COLOR_GRAY2BGR)
  train_img=np.pad(train_img,((r,r),(r,r),(0,0)),'constant',constant_values=(0,0))
  train_img=train_img[:,:,0]
  #Norm
  lst_img = sample_normalization(lst_img)
  lst_img = (lst_img - np.mean(lst_img))
  X=[]
  Y=[]
  count=0
  #roi坐标打乱
  idx,idy=np.where(train_img!=0)
  ID = np.random.permutation(len(idx))#样本点打乱排序
  idx=idx[ID]
  idy=idy[ID]
  #莫名操作?
  # per=0.89
  # idx = idx[:int(per*len(idx))]
  # idy = idy[:int(per*len(idy))]
  print('获取roi_patches中')
  for i in tqdm(range(len(idx))):
    tmp_lst=lst_img[idx[i]-r:idx[i]+r+1,idy[i]-r:idy[i]+r+1,:]
    tmpy=train_img[idx[i], idy[i]]
    if (tmpy == 3):
      tmpy=1
    else:
      tmpy=0
    X.append(tmp_lst)
    Y.append(tmpy)
    count+=1
  X = np.asarray(X, dtype=np.float32)
  Y = np.asarray(Y, dtype=np.int8)
  return X,Y

In [ ]:
landsat_img_path='/content/gdrive/MyDrive/CM/img/ResearchAreaTIF.tif'
train_img_path='/content/gdrive/MyDrive/CM/img/train_labels.tif'

In [ ]:
origin=read_image(landsat_img_path)

In [ ]:
origin.shape

(7921, 7801, 7)

In [ ]:
import random
import itertools
n=1000#随机样本数
def create_unlabeled_tif(tifshape,n=n):
  height=tifshape[0]
  width=tifshape[1]
  unlabeled=np.zeros((height,width))
  random_list = list((itertools.product(range(1,height),range(1,width))))#所有组合
  random_point=random.sample(random_list, n)
  
  for i,j in random_point:
    unlabeled[i,j]=1
  return unlabeled

In [ ]:
unlabeled_sample=create_unlabeled_tif(origin.shape)

In [ ]:
tiff.imsave('/content/gdrive/MyDrive/CM/unlabeled_patches/unlabeled_sample.tif',unlabeled_sample)

In [ ]:
unlabeled_path='/content/gdrive/MyDrive/CM/unlabeled_patches/unlabeled_sample.tif'

In [ ]:
X,Y=create_patches(landsat_img_path,unlabeled_path)

In [ ]:
X,Y=create_patches(landsat_img_path,train_img_path)

In [ ]:
np.save(save_path+'/'+'unlabeled_X.npy', X)
# np.save(save_path+'/'+'test_Y_new.npy', Y)

In [ ]:
X_train_path = '/content/gdrive/MyDrive/CM/roi_patches/test_X_new.npy'
Y_train_path = '/content/gdrive/MyDrive/CM/roi_patches/test_Y_new.npy'

In [ ]:
X_train = tf.keras.utils.to_categorical(np.load(X_train_path))
Y_train = tf.keras.utils.to_categorical(np.load(Y_train_path))

In [ ]:
X_train.shape

(454, 11, 11, 7, 3)